<a href="https://colab.research.google.com/github/aavarela/SPBD_Labs/blob/main/Lab4_exercises_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Spark Exercises


In [ ]:
#@title Download "Os Maias"
!wget -q -O os_maias.txt https://www.dropbox.com/s/n24v0z7y79np319/os_maias.txt?dl=0
!pip install --quiet unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.7 MB/s eta 0:00:00


##1. Word Frequency Exercises

1.1) Create a [Spark Core](https://spark.apache.org/docs/latest/api/python/) program that counts the number of occurrences of each word in the novel "Os Maias":

a) No sorting required, words and frequency can appear in any order;

b) Sorted by word, in reverse alphabetical order;

c) Sorted by frequency (the words with higher occurrence first).

Note that the sorting should be performed as a transformation (i.e. it should produce an RDD)...

In [ ]:
#@title WordFrequency Spark Core 1.1a)
import pyspark, string
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from unidecode import unidecode

spark = SparkSession \
    .builder \
    .appName("WordFrequency1.1a") \
    .getOrCreate()

sc = spark.sparkContext

try:
  lines = sc.textFile('os_maias.txt') \
      .map( lambda line: line.strip() ) \
      .map( lambda line: unidecode(line).lower() ) \
      .map( lambda line: line.translate(str.maketrans('', '', string.punctuation+'«»')) ) \

  words = lines.flatMap( lambda line: line.split() ) \
          .map( lambda word: (word, 1)) \
          .reduceByKey( lambda a, b: a+b)


  for w,f in words.take(20):
      print("{}\t{}".format(w,f))

  sc.stop()
except Exception as e:
  print(e)
  sc.stop()


eca	1
capitulo	20
casa	327
no	1569
outono	21
na	1320
da	2237
rua	163
paula	2
o	7581
bairro	21
das	480
verdes	14
apesar	40
nome	112
sombrio	24
casarao	8
paredes	19
severas	3
um	3191


In [ ]:
#@title WordFrequency Spark Core 1.1b)
import pyspark, string
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from unidecode import unidecode

spark = SparkSession \
    .builder \
    .appName("WordFrequency1.1b") \
    .getOrCreate()

sc = spark.sparkContext

try:
  lines = sc.textFile('os_maias.txt') \
      .map( lambda line: line.strip() ) \
      .map( lambda line: unidecode(line).lower() ) \
      .map( lambda line: line.translate(str.maketrans('', '', string.punctuation+'«»')) ) \

  words = lines.flatMap( lambda line: line.split() ) \
          .map( lambda word: (word, 1)) \
          .reduceByKey( lambda a, b: a+b)

  sorted_words = words.sortByKey(ascending=False)

  for w,f in sorted_words.take(20):
      print("{}\t{}".format(w,f))

  sc.stop()
except Exception as e:
  print(e)
  sc.stop()

zurzira	1
zumzum	1
zumbir	1
zuavos	1
zombarias	1
zola	3
zinco	3
zeloso	1
zelosamente	1
zelo	1
zelava	1
zelandia	1
zeferino	1
zas	7
zangues	2
zangouse	2
zangava	1
zangase	1
zangarse	1
zangar	3


In [ ]:
#@title WordFrequency Spark Core 1.1c)
import pyspark, string
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from unidecode import unidecode

spark = SparkSession \
    .builder \
    .appName("WordFrequency1.1c") \
    .getOrCreate()

sc = spark.sparkContext

try:
  lines = sc.textFile('os_maias.txt') \
      .map( lambda line: line.strip() ) \
      .map( lambda line: unidecode(line).lower() ) \
      .map( lambda line: line.translate(str.maketrans('', '', string.punctuation+'«»')) ) \

  words = lines.flatMap( lambda line: line.split() ) \
          .map( lambda word: (word, 1)) \
          .reduceByKey( lambda a, b: a+b)

  sorted_words = words.sortBy(lambda t : t[1], ascending=False)

  for w,f in sorted_words.take(20):
      print("{}\t{}".format(w,f))

  sc.stop()
except Exception as e:
  print(e)
  sc.stop()

de	8464
a	8136
o	7581
e	7526
que	5309
um	3191
com	2872
do	2592
uma	2298
da	2237
nao	2169
os	1888
para	1822
carlos	1798
as	1694
em	1626
no	1569
se	1534
ao	1487
na	1320


1.2) Create a Spark program that the top 10 most used words in "Os Maias" novel.

You should try to avoid sorting or finding the top-10 as actions. Your top-10 most used words should be a RDD at the end of the computation. Check *zipWithIndex* in [pyspark RDD](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html#pyspark.RDD) documentation

In [ ]:
#@title 1.2) Top10Words Version 1
import pyspark, string
from pyspark.sql import SparkSession
from unidecode import unidecode

spark = SparkSession \
    .builder \
    .appName("Top10Words 1.2) v1") \
    .getOrCreate()

sc = spark.sparkContext

try:
  lines = sc.textFile('os_maias.txt') \
      .map( lambda line: line.strip() ) \
      .map( lambda line: unidecode(line).lower() ) \
      .map( lambda line: line.translate(str.maketrans('', '', string.punctuation+'«»')) ) \


  top10_words_partitions = lines.flatMap( lambda line: line.split() ) \
          .map( lambda word: (word, 1)) \
          .reduceByKey( lambda a, b: a + b) \
          .mapPartitions( lambda partition : sorted(partition, key=lambda kv: kv[1], reverse=True)[0:10])

  top10_words = top10_words_partitions.sortBy(lambda x: x[1], ascending = False) \
                .zipWithIndex() \
                .filter( lambda ranked: ranked[1] < 10) \
                .map( lambda ranked: ranked[0])

  print("Partitions: Top-10")
  for x in top10_words_partitions.glom().collect():
      print("{}".format(x))

  print("Top-10 Most frequent words:")
  for x in top10_words.collect():
      print("{}".format(x))

except Exception as e:
  print(e)


Partitions: Top-10
[('o', 7581), ('um', 3191), ('uma', 2298), ('da', 2237), ('para', 1822), ('as', 1694), ('no', 1569), ('ao', 1487), ('na', 1320), ('por', 1094)]
[('de', 8464), ('a', 8136), ('e', 7526), ('que', 5309), ('com', 2872), ('do', 2592), ('nao', 2169), ('os', 1888), ('carlos', 1798), ('em', 1626)]
Top-10 Most frequent words:
('de', 8464)
('a', 8136)
('o', 7581)
('e', 7526)
('que', 5309)
('um', 3191)
('com', 2872)
('do', 2592)
('uma', 2298)
('da', 2237)


In [ ]:
#@title 1.2) Top10Words Version 2
import pyspark, string
from pyspark.sql import SparkSession
from unidecode import unidecode

spark = SparkSession \
    .builder \
    .appName("Top10Words 1.2) v2") \
    .getOrCreate()

sc = spark.sparkContext

try:
  lines = sc.textFile('os_maias.txt') \
      .map( lambda line: line.strip() ) \
      .map( lambda line: unidecode(line).lower() ) \
      .map( lambda line: line.translate(str.maketrans('', '', string.punctuation+'«»')) ) \

  top10_words_partitions = lines.flatMap( lambda line: line.split() ) \
          .map( lambda word: (word, 1)) \
          .reduceByKey( lambda a, b: a+b) \
          .glom() \
          .map( lambda partition : sorted(partition, key=lambda kv: kv[1], reverse=True)[0:10]) \
          .flatMap( lambda _ : _ )

  top10_words = top10_words_partitions.sortBy(lambda x: x[1], ascending = False) \
                .zipWithIndex() \
                .filter( lambda ranked: ranked[1] < 10) \
                .map( lambda ranked: ranked[0])

  print("Partitions: Top-10")
  for x in top10_words_partitions.glom().collect():
      print("{}".format(x))

  print("Top-10 Most frequent words:")
  for x in top10_words.collect():
      print("{}".format(x))

except Exception as e:
  print(e)

Partitions: Top-10
[('o', 7581), ('um', 3191), ('uma', 2298), ('da', 2237), ('para', 1822), ('as', 1694), ('no', 1569), ('ao', 1487), ('na', 1320), ('por', 1094)]
[('de', 8464), ('a', 8136), ('e', 7526), ('que', 5309), ('com', 2872), ('do', 2592), ('nao', 2169), ('os', 1888), ('carlos', 1798), ('em', 1626)]
Top-10 Most frequent words:
('de', 8464)
('a', 8136)
('o', 7581)
('e', 7526)
('que', 5309)
('um', 3191)
('com', 2872)
('do', 2592)
('uma', 2298)
('da', 2237)
